In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

### Importazione dataframe

In [30]:
df1 = pd.read_csv('C:/Users/HP/Desktop/PLC_gualchierani/monitorconi_23mar_to_29mar.csv')
df2 = pd.read_csv('C:/Users/HP/Desktop/PLC_gualchierani/monitorconi_30mar_to_4apr.csv')
df3 = pd.read_csv('C:/Users/HP/Desktop/PLC_gualchierani/monitorconi_5apr_to_11apr.csv')
df4 = pd.read_csv('C:/Users/HP/Desktop/PLC_gualchierani/monitorconi_12apr_to_18apr.csv')
df5 = pd.read_csv('C:/Users/HP/Desktop/PLC_gualchierani/monitorconi_19apr_to_25apr.csv')
df6 = pd.read_csv('C:/Users/HP/Desktop/PLC_gualchierani/monitorconi_26apr_to_2mag.csv')
df7 = pd.read_csv('C:/Users/HP/Desktop/PLC_gualchierani/monitorconi_3mag_to_9mag.csv')

In [31]:
df1["measured_at"] = pd.to_datetime( df1["measured_at"] )
df2["measured_at"] = pd.to_datetime( df2["measured_at"] )
df3["measured_at"] = pd.to_datetime( df3["measured_at"] )
df4["measured_at"] = pd.to_datetime( df4["measured_at"] )
df5["measured_at"] = pd.to_datetime( df5["measured_at"] )
df6["measured_at"] = pd.to_datetime( df6["measured_at"] )
df7["measured_at"] = pd.to_datetime( df7["measured_at"] )

In [32]:
mask_30mar = np.asarray( df1[ df1['measured_at'].dt.day == 30].index )
df1.drop( mask_30mar, inplace = True )

mask_29mar = np.asarray( df2[ df2['measured_at'].dt.day == 29].index )
df2.drop( mask_29mar, inplace = True )

mask_5apr = np.asarray( df2[ df2['measured_at'].dt.day == 5].index )
df2.drop( mask_5apr, inplace = True )

mask_4apr = np.asarray( df3[ df3['measured_at'].dt.day == 4].index )
df3.drop( mask_4apr, inplace = True )

mask_12apr = np.asarray( df3[ df3['measured_at'].dt.day == 12].index )
df3.drop( mask_12apr, inplace = True )

mask_11apr = np.asarray( df4[ df4['measured_at'].dt.day == 11].index )
df4.drop( mask_11apr, inplace = True )

mask_19apr = np.asarray( df4[ df4['measured_at'].dt.day == 19].index )
df4.drop( mask_19apr, inplace = True )

mask_18apr = np.asarray( df5[ df5['measured_at'].dt.day == 18].index )
df5.drop( mask_18apr, inplace = True )

mask_26apr = np.asarray( df5[ df5['measured_at'].dt.day == 26].index )
df5.drop( mask_26apr, inplace = True )

mask_25apr = np.asarray( df6[ df6['measured_at'].dt.day == 25].index )
df6.drop( mask_25apr, inplace = True )

mask_3mag = np.asarray( df6[ df6['measured_at'].dt.day == 3].index )
df6.drop( mask_3mag, inplace = True )

mask_2mag = np.asarray( df7[ df7['measured_at'].dt.day == 2].index )
df7.drop( mask_2mag, inplace = True )

mask_10mag = np.asarray( df7[ df7['measured_at'].dt.day == 10].index )
df7.drop( mask_10mag, inplace = True )

df = pd.concat([df1, df2, df3, df4, df5, df6, df7])

In [33]:
df.drop(['id', 'tipologia_lavorazione', 'cicli_da_eseguire',
        'cicli_eseguiti', 'tempo_miscelazione_eseguito',
        'stato_tramoggia', 'tara_cono', 'lotto',
        'created_at', 'updated_at', 'tempo_miscelazione_richiesto', 'destinazione_ritorno'], axis=1, inplace=True)

In [34]:
mask_no_cono = np.asarray( df[ df['id_cono'] > 24 ].index )
df.drop( mask_no_cono, inplace = True )

In [35]:
len(df)

1338402

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1338402 entries, 0 to 528503
Data columns (total 9 columns):
 #   Column                            Non-Null Count    Dtype         
---  ------                            --------------    -----         
 0   id_cono                           1338402 non-null  int64         
 1   ubicazione                        1338402 non-null  int64         
 2   stato                             1338402 non-null  int64         
 3   postazione_pesatura_destinazione  1338402 non-null  int64         
 4   miscelatore_destinazione          1338402 non-null  int64         
 5   estrusore_destinazione            1338402 non-null  int64         
 6   tempo_miscelazione_richiesto      1338402 non-null  int64         
 7   destinazione_ritorno              1338402 non-null  int64         
 8   measured_at                       1338402 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(8)
memory usage: 102.1 MB


### Preparazione dati di supporto

In [88]:
buffer = [n for n in range(21, 29)]
buffer.extend([n for n in range(31, 39)])

legenda_gualchierani = {
    'fuori_impianto': [0],
    'ing_zona_misc': [1],
    'catenaria_ing_misc': [45, 46],
    'navetta_serv_misc': [43],
    'pesatura': [41, 42],
    'miscelatore': [2, 8],
    'cat_usc_pieni': [3],
    'nav_serv_carr_pieni': [4],
    'nav_serv_carr_vuoti': [5],
    'buffer': buffer,
    'carroponte_1': [6],
    'carroponte_2': [7],
    'estrusore': [n for n in range(11, 19)],
    'handling_post_est': [6, 5, 4],
    'catenaria_vuoti': [47, 48],
    'catenaria_uscita_vuoti': [49],
    'pinza_vuoti': [50],
    'catenaria_parcheggi': [n for n in range(51, 58)],
    'nav_serv_parcheggi': [44],
    'manuale': [68, 70]
}

dict_states = {
    'Not_def': 0,
    'V_per_park': 1,
    'P_da_misc': 2,
    'P_da_est': 3,
    'P_remove': 4,
    'P_recheck': 5,
    'P_per_park': 6,
    'V_da_pes': 7,
    'V_remove': 8,
    'V_lavaggio': 9
}

starter = ['V_da_pes/nav_serv_carr_pieni', 'V_da_pes/nav_serv_carr_vuoti',
'V_da_pes/catenaria_vuoti', 'V_da_pes/catenaria_parcheggi',
'V_da_pes/nav_serv_parcheggi', 'V_da_pes/catenaria_ing_misc',
'V_da_pes/ing_zona_misc']

finisher = ['V_per_park/estrusore', 'V_da_pes/estrusore', 'V_remove/estrusore',
            'V_lavaggio/estrusore', 'P_da_est/fuori_impianto']

### Preparazione funzioni

In [38]:
df.reset_index(drop=True, inplace=True)

In [41]:
def naming_values(df, legenda_gualchierani, dict_states):
    for idx in df.index:
        for key in legenda_gualchierani:
            if df.loc[idx, 'ubicazione'] in legenda_gualchierani[key]:
                df.loc[idx, 'ubicazione'] = key
                break
        for key in dict_states:
            if df.loc[idx, 'stato'] == dict_states[key]:
                df.loc[idx, 'stato'] = key
                break
    return(df)

In [42]:
df = naming_values(df, legenda_gualchierani, dict_states)

In [45]:
#  crea il dataframe che sarà input alla funzione di individuazione della tracce
def create_rif(df):
    df['ID'] = 0
    df['CRC'] = 0
    df['rif'] = (df.stato.astype('str') + '/' + df.ubicazione.astype('str'))
    return(df)

In [46]:
df = create_rif(df)

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338402 entries, 0 to 1338401
Data columns (total 12 columns):
 #   Column                            Non-Null Count    Dtype         
---  ------                            --------------    -----         
 0   id_cono                           1338402 non-null  int64         
 1   ubicazione                        1338402 non-null  object        
 2   stato                             1338402 non-null  object        
 3   postazione_pesatura_destinazione  1338402 non-null  int64         
 4   miscelatore_destinazione          1338402 non-null  int64         
 5   estrusore_destinazione            1338402 non-null  int64         
 6   tempo_miscelazione_richiesto      1338402 non-null  int64         
 7   destinazione_ritorno              1338402 non-null  int64         
 8   measured_at                       1338402 non-null  datetime64[ns]
 9   ID                                1338402 non-null  int64         
 10  CRC               

In [74]:
#  questa funzione individua delle potenziali tracce di processo
#  prende in input una lista di possibili punti di inizio ed una di possibili punti di fine

def delete_duplicates(df, cono_in_esame):
    cono = []
    rif = []
    timestamp = []
    df_c = df[df['id_cono'] == cono_in_esame].copy()
    df_c.reset_index(inplace=True, drop=True)
    for idx in range(len(df_c.index) - 1):
        if idx == 0 or idx == len(df_c):
            cono.append(cono_in_esame)
            rif.append(df_c.loc[idx, 'rif'])
            timestamp.append(df_c.loc[idx, 'measured_at'])
        elif (df_c.loc[idx, 'rif'] != df_c.loc[idx - 1, 'rif']
            and df_c.loc[idx, 'rif'] == df_c.loc[idx + 1, 'rif']):
            cono.append(cono_in_esame)
            rif.append(df_c.loc[idx, 'rif'])
            timestamp.append(df_c.loc[idx, 'measured_at'])
        else:
            pass
    
    df_c = pd.DataFrame(list(zip(cono, rif, timestamp)), columns =['cono', 'rif', 'timestamp'])
    df_c.sort_values('timestamp', inplace=True)
    return(df_c)

In [75]:
for cono in range(1, 25):
    print(cono)
    if cono == 1:
        df_clean = delete_duplicates(df, cono)
    else:
        df_clean = pd.concat([df_clean, delete_duplicates(df, cono)])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [76]:
df_clean

,cono,rif,timestamp
0,1,P_da_est/navetta_serv_misc,2022-03-23 00:00:29
1,1,P_da_est/nav_serv_carr_pieni,2022-03-23 00:01:38
2,1,P_da_est/carroponte_1,2022-03-23 00:03:21
3,1,P_da_est/buffer,2022-03-23 00:05:38
4,1,P_da_est/estrusore,2022-03-23 00:09:04
...,...,...,...
5113,23,V_da_pes/pesatura,2022-05-09 23:59:15
0,24,P_da_est/fuori_impianto,2022-03-23 00:00:29
1,24,P_per_park/fuori_impianto,2022-04-27 01:53:16
2,24,V_per_park/fuori_impianto,2022-04-27 02:13:18


### Calcolo tracce

In [100]:
def calculate_traces(df_clean, finisher):
    df_clean['ID_unique'] = None
    
    for cono in range(1, 25):
        trace = 0
        df_c = df_clean[df_clean['cono'] == cono].copy()
        df_c.sort_values('timestamp', inplace=True)
        df_c.reset_index(inplace=True, drop=True)
        start = False
        end = True
        for idx in df_c.index:
            if df_c.loc[idx, 'rif'] == 'V_da_pes/pesatura' and not start and end:
                df_c.loc[idx, 'ID_unique'] = str(trace) + 'C' + str(cono)
                start = True
                end  = False
            elif df_c.loc[idx, 'rif'] in finisher and start and not end:
                df_c.loc[idx, 'ID_unique'] = str(trace) + 'C' + str(cono)
                start = False
                end  = True
                trace = trace + 1
            elif start and not end:
                df_c.loc[idx, 'ID_unique'] = str(trace) + 'C' + str(cono)
            else:
                pass
        if cono == 1:
            df_traces = df_c.copy()
        else:
            df_traces = pd.concat([df_traces, df_c])
    return(df_traces)

In [101]:
df_traces = calculate_traces(df_clean, finisher)

In [102]:
df_traces.dropna(inplace=True)

,cono,rif,timestamp,ID_unique
11,1,V_da_pes/pesatura,2022-03-23 01:54:52,0C1
12,1,P_da_misc/pesatura,2022-03-23 02:09:10,0C1
13,1,P_da_misc/miscelatore,2022-03-23 02:11:28,0C1
14,1,P_da_est/miscelatore,2022-03-23 02:13:45,0C1
15,1,P_da_est/cat_usc_pieni,2022-03-23 02:14:54,0C1
...,...,...,...,...
5106,23,P_da_est/carroponte_1,2022-05-09 22:48:52,393C23
5107,23,P_da_est/buffer,2022-05-09 22:51:10,393C23
5108,23,P_da_est/estrusore,2022-05-09 23:05:28,393C23
5109,23,V_per_park/estrusore,2022-05-09 23:48:57,393C23


In [110]:
df_traces.to_csv('C:/Users/HP/Desktop/df_traces_from_weigh.csv')

In [108]:
len(df_traces['ID_unique'].unique())

1620

In [109]:
df_traces.head(20)

,cono,rif,timestamp,ID_unique
11,1,V_da_pes/pesatura,2022-03-23 01:54:52,0C1
12,1,P_da_misc/pesatura,2022-03-23 02:09:10,0C1
13,1,P_da_misc/miscelatore,2022-03-23 02:11:28,0C1
14,1,P_da_est/miscelatore,2022-03-23 02:13:45,0C1
15,1,P_da_est/cat_usc_pieni,2022-03-23 02:14:54,0C1
16,1,P_da_est/nav_serv_carr_pieni,2022-03-23 02:18:19,0C1
17,1,P_da_est/carroponte_1,2022-03-23 02:20:02,0C1
18,1,P_da_est/estrusore,2022-03-23 02:22:20,0C1
19,1,V_per_park/estrusore,2022-03-23 03:06:55,0C1
24,1,V_da_pes/pesatura,2022-03-23 04:22:27,1C1
